# 1. Libraries

In [1]:
### Computational Libraries
import numpy as np
import pandas as pd

### HTML Servlet related
import requests
import markdown2
from xml.dom.minidom import parseString


### News Summary Library
import newspaper

### NLTK
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

### String Manipulation
from collections import defaultdict
from string import punctuation
from heapq import nlargest

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#### From Markdown to HTML

from webbrowser import open_new_tab


### Local HTML to PDF

import pdfcrowd
import sys

## Date for convention

from datetime import date
import calendar

# 2. Defined Classes

## 2.1 Class 'FrequencySummarizer' for Summarizing a Text

In [2]:
class FrequencySummarizer:
  def __init__(self, min_cut=0.5, max_cut=0.95):
    """ Words that have a frequency term lower than min_cut or higer than max_cut will be ignored. """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Input: a list of sentences already tokenized.
      Output:  a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """"
    Return a list of n sentences which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)

# 3. Web Search

*  The Data will be collected from the following News Sources only via Google-News:
    - goal.com  [HomePage](https://www.goal.com/en)
    - skysports.com  [HomePage](https://www.skysports.com/football)
    - espn.in  [HomePage](http://www.espn.in/football/?src=com)
    - theguardian.com  [HomePage](https://www.theguardian.com/football)


*  The News window is set to past 7 days.

- This is not an exhaustive list. You can customize this list by adding/removing websites. You can also filter particluar websited that you don't want in your search list.

- News wrt **Liverpool** and **English Premier League** will be fetched.
- News wrt **Liverpool** and **Champions League** will also be fetched separately.
- Then, I will combine both the News sources to only look for unique News.

# 4. News Compilation

In [3]:
urls = 0    ####Initialisation of the urls variable

sites = ['goal.com', 'skysports.com', 'espn.in', 'theguardian.com/football']    #### List to store the Websites

## 4.1 Function to fetch the URL and Title.

In [4]:
def URL_Decorator( func):
    """
    *  The function searches for the news as entered in the term 'search_string'
    *  The function also returns the title of the News for the fetched URL - retured by 'return_title' """
    def wrapper(string, days, urls):
        """ A Function Wrapper """
        term, count = func(string,days, urls)
        results = []
        results_title = []
        results_url = []
        obj = parseString( requests.get('https://news.google.com/rss/search?q={0}&hl=en-IN&gl=IN&ceid=IN:en'.format(term)).text )
        items = obj.getElementsByTagName('item') # Get each item
        for item in items[:count]:
            t,l = '', ''
            for node in item.childNodes:
                if node.nodeName == 'title':
                    t = node.childNodes[0].data
                elif node.nodeName == 'link':
                    l = node.childNodes[0].data
            results.append( (t,l) )
            results_title.append(t)
            results_url.append(l)
        return results_title, results_url
        
    return wrapper
    

@URL_Decorator
def search_strings(search_string, days, url):
    """
    A Function to write automated script as an input for 'get_google_new_results_test' function.
    
    search_string = Value that we need to search
    days = No of Days in the past
    urls = No of URL's to fetch
    """
    urls = int(url)
    for site in sites:
        if site == sites[0]:
            string = str('{0} {1}').format(search_string, site)
        elif site != sites[-1]:
            string = str(string) + str(' OR ') + str('{0}').format(site)
        else:
            string = string + str(' when:{0}d').format(days)
    return string, urls

# 7. Scraping Data into Markdown File

In [5]:
## Defining the Class Object ##
fs = FrequencySummarizer()

In [6]:
### Adding Naming Convension - based on Date
today_date = date.today()
today_date

day_file_name = str(today_date.day)+'/'+today_date.strftime('%m')+'/'+str(today_date.year)   ## In case Publish Date is absent
file2 = str(today_date.day)+'_'+today_date.strftime('%m')+'_'+str(today_date.year)
file_name = file2 + '_Report.pdf'

final_file_name = str(file_name)

In [7]:
%%capture output

del total_urls
total_urls = []
### Setting search term
search_term = ['Liverpool AND English Premier League', 'Liverpool AND Champions League']


headings = ['English Premier League', 'Champions League']    #### Setting Headings

for term, heading in zip(search_term,headings):
    
    title, urls = search_strings(('{0}').format(term), 20, 10)
    # only B = B - set(A UNION B)
    urls =  list(set(urls)^set(list(set(urls).intersection(total_urls))))  ## Only Unique URL's with respect to that fruit
    total_urls.extend(urls)
    
    print('<center><h3>' + str('{0}  Overview').format(str(heading))+'</h3></center>')

    for url in urls:
        article = newspaper.Article(url)
        try:
            article.download()
            article.parse()
        except:
            continue



        title_url = article.title
        text_text = article.text

        try:
            if(int(article.publish_date.month)>10):

                publish_date = str(article.publish_date.day)+'/' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
                publish_date = str(publish_date)
            else:
                publish_date = str(article.publish_date.day)+'/0' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
                publish_date = str(publish_date)
        except AttributeError:
            publish_date = str(day_file_name)


            ## Summarization of a News Column ##


        print('\n')
        print('<table>')

        print('<tr>')
        print('<th>')

        #print('[' + str(title_url)+ ']' + '('+ str(url) + ')' )
        print('<a href = "{0} ">'.format(url)+ str(title_url) +'</a>')
        #print(Date {0}.format(publish_date))

        #print('</div>')

        print('</th>')
        print('</tr>')
        print('</table>')              
            #print('----------------------------------')
        print('\n')
        print('\n')
        print('<article><strong>'+format(publish_date) + '</article>')


        try:

            for s in fs.summarize(text_text,5):
                    print('*  ',s)


            print('\n')
            #print('<center>')
            #print('\n')
            #print('[Go to HeadLines]' +'('+str(url)+')')
            #print('\n')
            #print('</center>')
                #print('---')  
        except AssertionError:
            continue

        print('\n')


                  
    print('\n')

In [8]:
### Getting Text of the Output
Summary = output.stdout

In [9]:
## Direct Markdown conversion of Summary in 'test_summary'
test_summary = Markdown(data= Summary)
test_summary

<center><h3>English Premier League  Overview</h3></center>


<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/salah-nets-50th-premier-league-goal-for-liverpool-in-record/1hjzo62tbsadb15n056ih1cqxt ">Liverpool News: Mohamed Salah scores 50th Premier League goal for club in record time</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
*   Salah nets 50th Premier League goal for Liverpool in record time

The Reds Egyptian forward netted his half-century of goals for the club faster than anyone else, beating former Anfield star Torres

Mohamed Salah became the fastest player to reach 50 Premier League goals for as he helped fire the Reds back to the top of the Premier League on Friday.
*   💪🔴 pic.twitter.com/1k89J6i8xT — Liverpool FC (@LFC) April 5, 2019

Luis Suarez, the star of Liverpool's last serious title challenge in 2013-14, took 86 matches to score 50 Premier League goals for the club, while Robbie Fowler needed 88 games.
*   Liverpool had trailed to Shane Long's early opener before Naby Keita equalised with his first Premier League goal.
*   Salah's brilliant breakaway goal ended a run of eight games without scoring in all competitions and also took him to a milestone, reaching a half-century of league strikes for Liverpool.
*   💫 RECORD BREAKER 💫



⚽ Games taken to net 50 @premierleague goals for the Reds…



6⃣9⃣ Salah

7⃣2⃣ Torres

8⃣6⃣ Suarez

8⃣8⃣ Fowler



Brilliant from @MoSalah .






<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/liverpool-forward-sadio-mane-shortlisted-for-premier-league/ol9od3hu0rdn174ohjkmah4ww ">Liverpool forward Sadio Mane shortlisted for Premier League Player of the Month</a>
</th>
</tr>
</table>




<article><strong>6/04/2019</article>
*   Liverpool forward Sadio Mane shortlisted for Premier League Player of the Month

The 26-year-old scored three league goals in four games in the month under review and he is set to be rewarded for the feat

forward Sadio Mane has been nominated for Premier League Player of the Month for March.
*   Article continues below

The 26-year-old will slug it out with teammate Andy Robertson, ’s Bernado Silva, ’s Jamie Vardy, ’s James Ward-Prowse and defender Seamus Coleman for the individual prize.
*   This season, the forward has scored 17 goals in the English elite division in 31 appearances and 20 in all competitions.
*   The international was in a brilliant form last month, scoring three league goals in four games, including his brace against to help the Reds continue their charge towards the league title.
*   The former Southampton man who was named the Reds Player of the Month on Thursday is also in contentions for the PFA award.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-in/news/live/transfer-news-and-rumours-live/ydgca4i9ccwu1mhmhtasfnp79 ">Transfer news and rumours LIVE: Liverpool looking to replace Mane with Depay</a>
</th>
</tr>
</table>




<article><strong>6/04/2019</article>


<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/man-city-stronger-liverpool-laporte/1rsy37jzjzo8911p56ollhsd1y ">Man City news: 'We are stronger than Liverpool' - Aymeric Laporte</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
*   "When you win almost every match, you feel like you're in a very big team, you have to keep going, try to win everything, that's what we want here," Laporte told Telefoot.
*   'Man City are stronger than Liverpool' - Laporte

The 24-year-old is determined to win every competition with Pep Guardiola's side and opened up on his situation with the France national team

defender Aymeric Laporte says his side are stronger than Premier League title rivals and is aiming to win every trophy he can with the club.
*   Article continues below

"I went through all the youth categories, I was captain, but the important thing today is the club and I am focused on that," he said.
*   And Laporte is confident his side can make the campaign a resounding success, hoping they end the season "in style".
*   Having already won the this term, Pep Guardiola's team are in contention to win a quadruple of trophies as they are gearing up for a quarter-final tie with and have booked a place in the final.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/premier-leagues-top-six-is-like-playing-bayern-munich-12/1gk87uh6qteqd1lgzlil4gxnml ">Southampton vs Liverpool: Premier League's top six is like playing Bayern Munich 12 times! - Ralph Hasenhuttl</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
*   That makes it a special challenge.”

On the collective quality of the Premier League being above that in the , Hasenhuttl added: “The top clubs in the Premier League benefit very much from the fact that there are six teams on a very high level.
*   - Hasenhuttl

The Southampton boss is readying his time for a meeting with Liverpool - the latest tough test he has faced since arriving in English football

manager Ralph Hasenhuttl, who arrived in English football from in December, claims facing the Premier League’s top six is like taking on 12 times.
*   are next on the agenda, with Hasenhuttl telling Goal and Spox of the elite competition he now finds himself up against: “We have six teams at the level of Bayern Munich in the Premier League.
*   Premier League's top six is like playing Bayern Munich 12 times!
*   And then we are the , or of the Premier League.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/liverpool-motivated-to-win-premier-league-says-sadio-mane/njdb4txacpjczvdbxz4uu8co ">Liverpool motivated to win Premier League, says Sadio Mane</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
*   Liverpool motivated to win Premier League, says Sadio Mane

The Senegal international has urged his side to win their last six games of the season to stand a chance of ending their title drought

Sadio Mane believes are well motivated to clinch the Premier League trophy this season.
*   “We are in two different competitions, the Premier League and Champions League, and we want to win both of them.
*   Our main target is to win the league and the Champions League because we have the quality in the team and the staff, the club is amazing.
*   “It doesn’t change anything; I will go there and try to give everything with my teammates to win there.”

Mane has scored 20 goals in all competitions this season including his 17 in the Premier League.
*   The Anfield outfit have failed to win a title since claiming the League Cup in 2012 and last won the English top-flight trophy in 1990.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-ie/news/west-ham-set-to-battle-bournemouth-for-permanent-clyne-deal/13y84neka71ik1bzbngotto5ft ">Liverpool transfer news: West Ham set to battle Bournemouth for permanent Nathaniel Clyne deal</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>
*   West Ham set to battle Bournemouth for permanent Clyne deal

The England full-back will depart Anfield this summer having impressed during a loan spell with the Cherries during the second half of this season

West Ham are ready to battle Bournemouth for the signing of defender Nathaniel Clyne this summer, Goal can reveal.
*   Goal understands that Clyne, who joined Liverpool from in 2015, has enjoyed his time at Bournemouth, where he has struck up a strong relationship with manager Eddie Howe.
*   The Cherries are keen to conclude a permanent deal once the transfer window re-opens, but will now face stiff competition from West Ham, who are keen to strengthen their full-back areas this summer.
*   Liverpool, though, have shown that they will not do business on the cheap.
*   And having sold the likes of Dominic Solanke and Danny Ings to Premier League clubs in the last 12 months, they are confident they can obtain a good fee for Clyne, who was competing for England’s right-back spot prior to his injury in the summer of 2017.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-bh/news/emery-exposed-as-everton-annihilate-arsenal-five-tactical/18p690u3ec67j1g8v3hth2yri1 ">Arsenal tactics: Unai Emery exposed by Everton & five tactical lessons from the weekend</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>






<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/eriksen-matches-beckhams-premier-league-record-as-tottenham-lose-/so650lmsefd7145f5lco7u1xd ">Liverpool 2-1 Tottenham: Eriksen matches Beckham's Premier League record as Spurs lose</a>
</th>
</tr>
</table>




<article><strong>31/03/2019</article>
*   The league title has long since slipped out of Tottenham’s reach, with Mauricio Pochettino’s team having now taken only one point from their last five league games

Spurs haven’t won since beating 3-1 on 10 February, a result that left them well in touch with the league’s top two.
*   Eriksen matches Beckham's Premier League record as Tottenham lose at Liverpool

The Dane set up Lucas Moura to equalise in the second half at Anfield before a late own-goal condemned Spurs to defeat

Christian Eriksen became just the second player to assist ten or more goals in four consecutive Premier League seasons during ’s 2-1 defeat to on Sunday.
*   Eriksen matches the record held by David Beckham who notched more than ten assists per season between 1997-8 and 2000-1, helping to three Premier League titles.
*   And with a resurgent Manchester United also stringing wins together under new full-time boss Ole Gunnar Solskjaer, Pochettino urgently needs to arrest his team’s spiral or face a first full season at the club’s new stadium without Champions League football.
*   10 - Christian Eriksen is just the second player to assist 10+ Premier League goals in four successive seasons, after David Beckham between 1997-98 and 2000-01.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-qa/news/liverpools-close-calls-doesnt-mentally-impact-manchester/1co0bnqecm93e1fxbpo0k5lsj3 ">Liverpool's close calls don't mentally impact Manchester City, says Pep Guardiola</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
*   Liverpool's close calls don't mentally impact Manchester City, says Pep Guardiola

The master coach believes the results of their title rivals do not have an impact on the team

manager Pep Guardiola insisted his quadruple-chasing team will not be psyched out by in the race for the Premier League trophy.
*   Guardiola's City are second in the Premier League, two points behind Liverpool but with a game in hand after the Reds claimed a late 3-1 win over on Friday.
*   I don't remember, I didn't see, but I think when Sir Alex Ferguson won the treble [with Manchester United in 1998-99], in that case not all the games were fantastic and winning four or five zero.
*   Asked the psychological impact on his players as Liverpool relentlessly pursue silverware despite a mixed run of recent form, Guardiola replied: "Zero.
*   Article continues below

City are also set to meet either or in the FA Cup decider next month and Guardiola added: "Do you believe we can play 60 games in a season and win 5-0 and have a good performance in every game?






<center><h3>Champions League  Overview</h3></center>


<table>
<tr>
<th>
<a href = "https://www.goal.com/en-qa/news/champions-league-2018-19-liverpool-porto-quarterfinals-sadio/r5bdb037dj2y1jlj1jqd65ito ">Sadio Mane has been the toast of the season for Liverpool</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>
*   His eye for goal has prevented Salah’s slump from affecting Liverpool’s chances in the league title race.
*   Sadio Mane has been the toast of the season for Liverpool

The Senegalese forward has taken the mantle from a struggling Mohamed Salah as the best player for Liverpool ...

start favourites as they engage FC in a quarter-final tie in the next week.
*   Both the players have breached the 20-goal mark this season in all competitions.
*   But Mane is clearly in better dynamics, peaking right at the business end of the season.
*   12 of his 21 goals this season has come in 2019.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-qa/news/champions-league-quarter-finals-fixtures-live-streams-how/1p2qpw2di0oma1rpgjgacw8jzb ">Champions League quarter-finals: Fixtures, live streams, how to watch & results</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>
*   April 9, 2019 - Tottenham vs Man City

UK TV channel: BT Sport 2

BT Sport 2 UK online stream: BT Sport app

BT Sport app US TV channel: Galavision / TNT USA

Galavision / TNT USA US online stream: Univision Deportes En Vivo / B/R Live / Watch TNT

April 9, 2019 - Liverpool vs Porto

UK TV channel: BT Sport 3

BT Sport 3 UK online stream: BT Sport app

BT Sport app US TV channel: Univision Deportes / UniMas

Univision Deportes / UniMas US online stream: B/R Live / Univision Now

April 10, 2019 - Man Utd vs Barcelona

UK TV channel: BT Sport 2

BT Sport 2 UK online stream: BT Sport app

BT Sport app US TV channel: Univision Deportes / UniMas / TNT USA

Univision Deportes / UniMas / TNT USA US online stream: Univision Deportes En Vivo / B/R Live / Watch TNT

April 10, 2019 - Ajax vs Juventus

UK TV channel: BT Sport 3

BT Sport 3 UK online stream: BT Sport app

BT Sport app US TV channel: Galavision

Galavision US online stream: B/R Live / Univision Deportes En Vivo

April 16, 2019 - Juventus vs Ajax

UK TV channel: BT Sport 3

BT Sport 3 UK online stream: BT Sport app

BT Sport app US TV channel: TBC

TBC US online stream: Univision Deportes En Vivo / B/R Live

April 16, 2019 - Barcelona vs Man Utd

UK TV channel: BT Sport 2

BT Sport 2 UK online stream: BT Sport app

BT Sport app US TV channel: TBC

TBC US online stream: B/R Live / Univision Deportes En Vivo

April 17, 2019 - Porto vs Liverpool

UK TV channel: BT Sport 3

BT Sport 3 UK online stream: BT Sport app

BT Sport app US TV channel: TBC

TBC US online stream: Univision Deportes En Vivo / B/R Live

April 17, 2019 - Man City vs Tottenham

UK TV channel: BT Sport 2

BT Sport 2 UK online stream: BT Sport app

BT Sport app US TV channel: TBC

TBC US online stream: B/R Live / Univision Deportes En Vivo

When are the Champions League semi-finals & final?
*   TV Channel, Live Stream & How To Watch

In the UK, Champions League quarter-final matches will be available to watch live on TV via BT Sport , who also offer an online streaming service through their app.
*   In the US, viewers are able to watch the Champions League quarter-final games on TV through the Univision network (which includes Univision Deportes, UniMas and Galavision) or TNT USA .






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/kevin-de-bruyne-dont-care-about-tottenham-stadium-manchester/isq1stnead8o1qh0a9p89aip1 ">Kevin De Bruyne: I don't care about Tottenham's stadium</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
*   De Bruyne insists he doesn't care about Tottenham's new stadium ahead of Champions League clash

The Manchester City playmaker stated he's more concerned about the Spurs team rather than the buzz surrounding their new ground

midfielder Kevin De Bruyne does not think being in their new stadium will have any significant impact upon Tuesday's quarter-final showdown.
*   Guardiola thinks City completing an unprecedented quadruple remains "almost impossible" and De Bruyne feels the gruelling schedule they have to negotiate should be acknowledged.
*   Article continues below

"I don't care about the stadium," he said.
*   Ahead of another trip to face a fired-up Premier League rival, De Bruyne was unequivocal over what lessons City can learn from that Merseyside ordeal.
*   A memorable atmosphere is anticipated for City's first-leg visit to north London but De Bruyne is more concerned about the quality players such as Harry Kane, Christian Eriksen and Dele Alli bring to the Tottenham ranks.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/man-city-stronger-liverpool-laporte/1rsy37jzjzo8911p56ollhsd1y ">Man City news: 'We are stronger than Liverpool' - Aymeric Laporte</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
*   "When you win almost every match, you feel like you're in a very big team, you have to keep going, try to win everything, that's what we want here," Laporte told Telefoot.
*   'Man City are stronger than Liverpool' - Laporte

The 24-year-old is determined to win every competition with Pep Guardiola's side and opened up on his situation with the France national team

defender Aymeric Laporte says his side are stronger than Premier League title rivals and is aiming to win every trophy he can with the club.
*   Article continues below

"I went through all the youth categories, I was captain, but the important thing today is the club and I am focused on that," he said.
*   And Laporte is confident his side can make the campaign a resounding success, hoping they end the season "in style".
*   Having already won the this term, Pep Guardiola's team are in contention to win a quadruple of trophies as they are gearing up for a quarter-final tie with and have booked a place in the final.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en/news/we-dominated-liverpool-for-110-minutes-maldini-reflects-on-cruel-/42bimr27gbj1kegqp2hgdtry ">Champions League news: 'We dominated Liverpool for 110 minutes' - Paolo Maldini reflects on 'cruel' 2005 Istanbul final</a>
</th>
</tr>
</table>




<article><strong>5/04/2019</article>
*   'We dominated Liverpool for 110 minutes' - Maldini reflects on 'cruel' 2005 Champions League final

The Italian legend says that the Rossoneri's famous defeat against the Reds was "cruel" and that it fuelled their desire for revenge two years later

Paolo Maldini insists that "dominated" the 2005 final against , despite losing the match after a dramatic penalty shootout.
*   Liverpool staged an unprecedented fightback in the second half, however, beginning with Steven Gerrard's powerful header that helped galvanise the club's supporters watching from the stands.
*   Dudek was again the saviour in the shootout as he saved Milan's final penalty from Andriy Shevchenko and the Reds basked in their fifth European Cup victory.
*   "Let’s say that playing in that [Athens] final was a bit of a gamble," the ex- international added.
*   Carlo Ancelotti's side raced into a 3-0 lead during a frenetic first half, with Maldini heading home the opening goal after just 50 seconds.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-qa/news/pogba-and-lukaku-are-capable-of-hurting-any-defence-coutinho/vv3ekinti1d1eab260zt0m78 ">Champions League news: ‘Pogba and Lukaku are capable of hurting any defence’ - Coutinho</a>
</th>
</tr>
</table>




<article><strong>7/04/2019</article>
*   ‘Pogba and Lukaku are capable of hurting any defence’ - Coutinho

The Barca star is gearing up to face his first Premier League opponent since leaving Liverpool during the 2017-18 season

Philippe Coutinho has said must be aware of the threat offered by ’s Paul Pogba and Romelu Lukaku when they travel to Old Trafford this week.
*   "We will have to be very careful to stop him because he can generate a lot of problems for us and I think the duel with [Gerard] Pique will be important in both matches.”

United head into the tie on the back of three losses in their last four, but Coutinho knows not to underestimate his old foes.
*   “It’s complicated to describe what you have to do to win against this team,” said the Brazilian, as per the Mirror.
*   “United have had some inconsistent results recently, but it’s clear they have improved in every area under [Ole Gunnar] Solskjaer.
*   “Let’s see, I hope it happens, but we have a big game first at Old Trafford,” he said.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-sg/news/netbusters-liverpool-uefa-champions-league-goals/e64k59mllycs1392jpd5vidm5 ">UEFA Champions League Netbusters: All Liverpool goals so far</a>
</th>
</tr>
</table>




<article><strong>25/03/2019</article>
*   UEFA Champions League Netbusters: All Liverpool goals so far

In this series, we take a look at all the goals scored by the remaining 8 teams in the Champions League....

Last year's finalists, , are certainly shaping well in the UEFA , with the competition set to enter the quarter-final phase.
*   Their forward line is a nightmare for any defence and they have scored 12 goals in the Champions League so far.
*   The Bavarians were beaten 3-1 on aggregate as Liverpool kept their bid to reach the final for a second consecutive year alive.
*   Liverpool now gear up to face FC in the quarter-final.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-kw/news/liverpool-can-easily-win-the-premier-league-the-champions-league-/urh9y2nw5z861b8mdzic4i8qf ">Liverpool news: Reds can easily win the Premier League & the Champions League, says Berger</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>
*   “It's great that if Salah is not scoring as much as he did last season there are other players that can score – and that's exactly what the boys need to do.”
*   I don't see why they can't win the double this season.”

Salah's strike rate of late has caused concern among fans after having had such a prolific campaign last year, but Berger is not overly concerned about the Egyptian's struggles due to squad depth and others stepping up to the plate.
*   “I think they can easily win the league and they can win the Champions League too,” the 45-year-old told Sky Sports.
*   In addition – despite Mohamed Salah's incredible goal return of 32 across the season – the Reds could only muster a fourth-place finish on domestic soil, but defensive reinforcements in Virgil van Dijk and Alisson have seen a porous back line improve significantly this term.
*   As a result, with his former side currently sitting at the summit of the Premier League and having drawn in Europe, whom Jurgen Klopp's side beat 5-0 on aggregate last season, Berger believes it is entirely possible that Liverpool can finally deliver for their fans.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/watch-innovation-champions-liverpools-well-rounded-defence-ready-/8xwitv008il51pvu3292ih970 ">WATCH: Innovation Champions - Liverpool's well-rounded defence ready to blossom in UEFA Champions League</a>
</th>
</tr>
</table>




<article><strong>3/04/2019</article>






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/pepe-real-madrid-was-a-centre-back-graveyard-before-i/4m31ttpdib5617nhugwqw2qo4 ">Real Madrid news: Pepe says Santiago Bernabeu was a centre-back graveyard before he arrived</a>
</th>
</tr>
</table>




<article><strong>8/04/2019</article>
*   Pepe: Real Madrid was a centre-back graveyard before I arrived

The outspoken defender believes there are more players on the books at Porto capable of following in his footsteps which led to the Spanish capital

Pepe claims were a “graveyard for centre-backs” prior to his arrival, with a man who thrived in now backing team-mate Eder Militao to follow in his footsteps.
*   He believes a solid grounding with Porto helped him to succeed, telling UEFA’s official website: “Porto improve every single player because this is a very demanding club.
*   “I came from a league viewed by many as not as competitive as others, but I played for 10 years and won many trophies.”

Pepe is now back at Porto and believes there a number of players currently on their books who would not look out of place in Madrid, with Militao having already agreed a €50m (£43m/$57m) summer switch.
*   !”

Before heading to Spain, Militao is looking to help Porto prolong a Champions League adventure in 2019, with Pepe and Co currently readying themselves for a quarter-final clash with Liverpool.
*   “Porto prepared me really well and this didn't go unnoticed in Spain, as before I went to Real Madrid the club were known as a graveyard for centre-backs.








### Conversion of Markdown to HTML

In [10]:
html = markdown2.markdown(str(Summary), html4tags=True, tab_width=5,  )

### Cleaning the HTML File

In [11]:
### Rectify Out of Bound Paras
html = html.replace("<p>","<ul><li><p>")

html = html.replace('</p>', '</p></li></ul>')

#### Rectify Headlines Link
html = html.replace("<ul><li><p><a href","<p><a href")

html = html.replace('/a></p></li></ul>', '/a></p>')

###  Rectify <centre>
html = html.replace("<ul><li><p><center></p></li></ul>","<p><center></p>")

html = html.replace('<ul><li><p></center></p></li></ul>', '<p></center></p>')


### Rectify *

html = html.replace("*","</li><li>")

### Rectify Image

html = html.replace("<li><img ","<lily imagine ")

#html = html.replace("</h1></center></p></li></ul>","</h1></center></p>")

## Rectify Title Bullet point

html = html.replace("<ul><li><p><center><h3>","<p><center><h3>")

html = html.replace("</h3></center></p></li></ul>","</h3></center></p>")

## Open in new tab

html = html.replace("<a href", '<a rel="noopener noreferrer" target="_blank" href')

### Checking the sample Webpage

In [13]:
filename = 'File.html'

f = open(filename,'w', encoding='utf-16')

wrapper = """<!DOCTYPE html>
<html lang="en">



<head>

<style>
.center {
  text-align: center;
  
}




</style>


<meta charset="ISO-8859-1" >
<div class="page center">
   <a href="https://www.liverpoolfc.com/welcome-to-liverpool-fc"><img class="auto-size__target postpone-load--fade-in widge-figure__image" data-src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" alt="" itemprop="image" src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" style="PADDING-RIGHT: 70px"></a>
</div>

<title>%s</title>

</head>

<body>
    
    %s
</body>



</html>"""

title = str('Liverpool News')

whole = wrapper % (title, html)

#whole = wrapper % (html)

f.write(whole)

f.close()

open_new_tab(filename)

True

## Conversion to PDF

In [70]:
try:
    # create the API client instance
    client = pdfcrowd.HtmlToPdfClient('<User_Name>', '<Secret_API_Key>')

    # run the conversion and write the result to a file
    client.convertFileToFile('File.html', final_file_name )
except pdfcrowd.Error as why:
    # report the error
    sys.stderr.write('Pdfcrowd Error: {}\n'.format(why))

    # handle the exception here or rethrow and handle it at a higher level
    raise